In [1]:
from difflib import SequenceMatcher
import pandas as pd
import numpy as np
import re
import pickle
from datetime import datetime
from pprint import pprint
from __future__ import division
import matplotlib.pyplot as plt 
import matplotlib
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from patsy import dmatrices
from patsy import dmatrix
from collections import defaultdict
from sklearn.linear_model import Lasso
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.special import factorial
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn import grid_search, datasets
from sklearn import linear_model
from sklearn.learning_curve import learning_curve
import math
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_curve, auc
from sklearn import tree
from sklearn import svm, grid_search
from sklearn.externals.six import StringIO
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime
import sklearn as sk
import pydot 
import os
import qgrid
from matplotlib.colors import ListedColormap
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.lda import LDA
from sklearn.qda import QDA
import requests
from bs4 import BeautifulSoup
import nltk
from textblob import TextBlob
from nltk.util import ngrams
from collections import defaultdict
from operator import itemgetter
from nltk.corpus import stopwords
from textblob.sentiments import NaiveBayesAnalyzer
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
import urllib2
import json
from pymongo import MongoClient
from fuzzywuzzy import process
%matplotlib inline 

In [2]:
with open('id_dict.pkl', 'r') as picklefile:
     id_dict = pickle.load(picklefile)
with open('cat_dict.pkl', 'r') as picklefile:
     cat_dict = pickle.load(picklefile)
with open('ex_df.pkl', 'r') as picklefile:
     ex = pickle.load(picklefile)
with open('ex2_df.pkl', 'r') as picklefile:
     ex2 = pickle.load(picklefile)
with open('ex3.pkl', 'r') as picklefile:
    ex3 = pickle.load(picklefile)
with open('pop_ids.pkl', 'r') as picklefile:
     pop_ids= pickle.load(picklefile)
with open('url_dict.pkl', 'r') as picklefile:
     url_dict = pickle.load(picklefile)
with open('test_urls.pkl', 'r') as picklefile:
     test_urls = pickle.load(picklefile)
with open('mini_dfs.pkl', 'r') as picklefile:
     minis = pickle.load(picklefile)

In [3]:
def get_restos(dataset, start=100001, num_dfs=50, top_x=10):
    df_dict = {}
    for i in range(start, (start + num_dfs*10000), 10000):
        df = dataset[i]
        df['name'] = df['business_id'].apply(lambda x: id_dict[x])
        df['cats'] = df['name'].apply(lambda x: cat_dict[x])
        df['CityName'] = df['business_id'].apply(lambda x: url_dict[x])
        df['restos'] = df['cats'].apply(lambda x: 1 if 'Restaurants' in x else 0)
        df = df[['name', 'cats', 'stars', 'text', 'date', 'user_id', 'restos', 'CityName']]
        url_list = df.pivot_table('stars', index='CityName', columns='restos', aggfunc='count', margins=True).sort(1, ascending=False)
        urls = url_list.reset_index()['CityName'].tolist()[1:top_x+1]
        df_dict[i] = urls, df
    return df_dict

In [187]:
def screlpy(cityname):
    url = get_url(cityname)
    if url:
        site = requests.get(url)
        page = site.text
        soup = BeautifulSoup(page)
        explore = soup.find(class_= 'menu-explore')
        if explore:
            url = 'http://www.yelp.com' + explore['href']
            site = requests.get(url)
            page = site.text
            soup = BeautifulSoup(page)
            main = soup.find(class_='menu-sections')
            menu = {}
            course_list = []
            options = []
            for items in main:
                courses = main.find_all(class_="menu-section-header")
                for course in courses:
                    course_list.append(course.find('h2').text.strip())
                sections = soup.find_all(class_ = 'menu-section')
                for section in sections:
                    food_list = []
                    foods = section.find_all('h3')
                    for food in foods:
                        food_list.append(food.text.strip())
                    options.append(food_list)

            menu = dict(zip(course_list, options))
        else:
            menu = 'Menu not on Yelp'
            print cityname + ' Menu not on Yelp'
    else:
        menu = 'Page Not Found'
    return menu

In [5]:
def get_url(cityname):
    cityname = cityname.replace(u'\xe9', 'e')
    cityname = cityname.replace(u'\xea', 'e')
    cityname = cityname.replace(u'\xe0', 'a')
    cityname = cityname.replace('&', 'and')
    cityname = cityname.replace('+', 'and')
    cityname = cityname.replace('---', '-')
    i = 0
    url = 'http://www.yelp.com/biz/'+ str(cityname)
    r = requests.get(url)
    try:
        url = 'http://www.yelp.com/biz/'+ str(cityname)
        r = requests.get(url)
    except:
        if (r.status_code != 200) & (i < 6):
            i += 1
            cityname = cityname + '-' + str(i)
        else:
            url = ''
    return url

In [6]:
def get_menus(citynames, menus=None):
    if not menus:
        menus = {}
    else:
        menus = menus
    for cityname in citynames:
        menus[cityname] = screlpy(cityname)
        cityname = cityname.replace(u'\xe9', 'e')
        cityname = cityname.replace(u'\xea', 'e')
        cityname = cityname.replace(u'\xe0', 'a')
        cityname = cityname.replace('&', 'and')
        cityname = cityname.replace('+', 'and')
        cityname = cityname.replace('---', '-')
        print 'screlped ' + str(cityname)
    return menus

In [243]:
def word_match(words, menu, vector_dict, threshold=75):
    matches = []
    for word in words:
        for key, value in menu.iteritems():
            for item in value:
                score = fuzz.ratio(word, item)
                if score > threshold:
                    matches.append((word, item, vector_dict[word]))
    if len(matches) < 5:
        for word in words:
            for key, course in menu.iteritems():
                for item in course:
                    short_item = ' '.join(item.split()[-2:])
                    score = fuzz.ratio(word, short_item)
                    if (score > threshold +5) and (word, item, vector_dict[word] not in matches):
                        matches.append((word, item, vector_dict[word]))
    if len(matches) < 5:
        for word in words:
            for key, course in menu.iteritems():
                for item in course:
                    short_item = ' '.join(item.split()[:2])
                    score = fuzz.ratio(word, short_item)
                    if (score > threshold +5) and (word, item, vector_dict[word] not in matches):
                        matches.append((word, item, vector_dict[word]))

    pprint(matches)
    return matches

In [235]:
def best_matches(ngrams, menu, vector_dict, threshold=70):
    best_matches = {}
    for ngram in ngrams:
        biggest = 0
        for key, value in menu.iteritems():
            for item in value:
                temp = {}
                score = fuzz.token_set_ratio(ngram, item)
                if score > threshold:
                    score2 = fuzz.partial_ratio(ngram, item)
                    score3 = fuzz.ratio(ngram, item)
                    temp[item] = score2+score3
                    if temp[item] > biggest:
                        biggest = temp[item]
                        best_matches[ngram] = item, vector_dict[ngram]
    if len(best_matches) < 5:
        for ngram in ngrams:
            biggest = 0
            for key, value in menu.iteritems():
                for item in value:
                    temp = {}
                    short_item = ' '.join(item.split()[-2:])
                    score = fuzz.token_set_ratio(ngram, short_item)
                    if score > threshold:
                        score2 = fuzz.partial_ratio(ngram, short_item)
                        score3 = fuzz.ratio(ngram, short_item)
                        temp[item] = score2+score3
                        if temp[item] > biggest:
                            biggest = temp[item]
                            best_matches[ngram] = item, vector_dict[ngram]
        
    pprint(best_matches)
    return best_matches

In [188]:
def best_matches2(ngrams, menu, vector_dict, threshold=75):
    best_matches = {}
    for key, course in menu.iteritems():
        for item in course:
            fuzz.extractBests(item, ngrams, scorer=fuzz.token_set_ratio, score_cutoff=threshold, limit=5)
            
            

In [10]:
def get_keyword_counts(column):
    text = column
    vectorizer = CountVectorizer(ngram_range=(1,3), stop_words='english')
    vectorizer.fit(text)
    features = vectorizer.get_feature_names()
    x = vectorizer.transform(text).toarray()
    counts = pd.DataFrame(x, columns=features)
    singles = counts.applymap(lambda x: 1 if x > 0 else 0)
    return singles

In [11]:
def get_weights(df, column=ex2['stars']):
    singles = df.copy().reset_index() 
    weighted = singles.join(column.reset_index(), lsuffix = '_0')
    totals = weighted.append(weighted.sum(numeric_only=True), ignore_index=True)
    dfs = {}
    for i in range(1,6):
        dfs[i] = weighted[weighted['stars']== i]
    return dfs

In [12]:
def get_totals(df_list):
    totals = {}
    for i in range(1,6):
        counts = {}
        df = df_list[i]
        for (index, value) in df.sum().iteritems():
            counts[index] = value
        totals[i] = counts
    return totals

In [13]:
def get_vectors(totals):
    vectors = {}
    for key in totals[1].keys():
        vectors[key] = sum([totals[1][key],totals[2][key],totals[3][key],totals[4][key],totals[5][key]]), totals[1][key],totals[2][key],totals[3][key],totals[4][key],totals[5][key]
    return vectors

In [208]:
def filter_keys(vector_dict):
    bigkeys = filter(lambda x: len(x.split()) > 1, vector_dict.keys())
    bigkeys = filter(lambda x: vector_dict[x][0] > 3, bigkeys)
    
    littlekeys = filter(lambda x: len(x.split()) == 1, vector_dict.keys())
    littlekeys = filter(lambda x: vector_dict[x][0] > 5, littlekeys)
    
    return littlekeys, bigkeys

In [215]:
def get_recs(vector_dict, menu):
    recs = []
    matches1 = word_match(filter_keys(vector_dict)[0], menu, vector_dict)
    matches2 = best_matches(filter_keys(vector_dict)[1], menu, vector_dict)
    for match in matches1:
        recs.append((get_scores(match[2]), match[0], match[1]))
    for key, value in matches2.iteritems():
        recs.append((get_scores(value[1]), key, value[0]))
    if recs:
        rdf = pd.DataFrame(recs)
        rdf['Score'] = rdf[0].apply(lambda x: x[1])
        rdf['Trendiness'] = rdf[0].apply(lambda x: x[0])
        rdf['Controversy'] = rdf[0].apply(lambda x: x[3])
        rdf['Buzz'] = rdf[0].apply(lambda x: x[2])
        rdf.rename(columns={1: 'Keyword', 2: 'MenuItem'}, inplace=True)
        rdf.drop(0, axis=1, inplace=True)
        rdf['Score'] = rdf['Score'].apply(lambda x: (x+2)*2.5)
        p_norm = rdf['Controversy'].max()
        b_norm = rdf['Trendiness'].max()
        rdf['Controversy'] = rdf['Controversy'].apply(lambda x: 10*x/p_norm)
        rdf['Trendiness'] = rdf['Trendiness'].apply(lambda x: 10*x/b_norm)
        return rdf.drop_duplicates()
    else:
        return pd.DataFrame()

In [16]:
def get_scores(vector):
    score = (2*vector[5] + vector[4] - vector[2] - 2*vector[1]) / vector[0]
    unweighted = (2*vector[5] + vector[4] - vector[2] - 2*vector[1])
    polarity = np.var(vector[5]*[5] + vector[4]*[4] + vector[3]*[3] + vector[2]*[2] + vector[1]*[1])
    buzz = vector[0]
    
    
    return unweighted, score, buzz, polarity

In [230]:
def final_recs(df):
    if not df.empty:
        rdf = df.groupby('MenuItem').agg({'Score': np.mean,'Trendiness': np.mean, 'Controversy': np.mean, 'Buzz': np.sum}).reset_index()
        b_norm = rdf['Trendiness'].max()
        rdf['Trendiness'] = rdf['Trendiness'].apply(lambda x: 10*np.log(x)/np.log(b_norm) if (x > 1) else 0)
        rdf['Trendiness'] = rdf['Trendiness'].apply(lambda x: np.round(x,3))
        rdf['Score'] = rdf['Score'].apply(lambda x: np.round(x,3))
        rdf['Controversy'] = rdf['Controversy'].apply(lambda x: np.round(x,3))
        
        return rdf.sort(['Trendiness', 'Score'], ascending=False)
    else:
        return 'No popular items =('

In [172]:
def recs_collector(db_dict, rec_dict=None, menu_list=None, start=100001, num_dfs=1):
    if not rec_dict:
        rec_dict = {}
    if not menu_list:
        menu_list = []
    for i in range(start, (start + num_dfs*10000), 10000):
        db = db_dict[i]
        menus = get_menus(db[0])
        ex = db[1].copy()
        for key, menu in menus.iteritems():
            if key not in rec_dict:
                if menu == 'Menu not on Yelp':
                    rec_dict[key] = 'Menu not on Yelp'
                    key = key.replace(u'\xe9', 'e')
                    key = key.replace(u'\xea', 'e')
                    key = key.replace(u'\xe0', 'a')
                    key = key.replace('&', 'and')
                    key = key.replace('+', 'and')
                    key = key.replace('---', '-')
                    print str(key) + "'s Menu not on Yelp =("
                else:
                    if key in ex['CityName'].unique():
                        df = ex[ex['CityName']==key]
                        counts = get_keyword_counts(df.text)
                        dfs = get_weights(counts, df['stars'])
                        totals = get_totals(dfs)
                        vectors = get_vectors(totals)
                        rdf = get_recs(vectors, menu)
                        final = final_recs(rdf)
                        rec_dict[key] = [final, rdf]
                        key = key.replace(u'\xe9', 'e')
                        key = key.replace(u'\xea', 'e')
                        key = key.replace(u'\xe0', 'a')
                        key = key.replace('&', 'and')
                        key = key.replace('+', 'and')
                        key = key.replace('---', '-')
                        print 'Rec Made on ' + str(key)
                    else:
                        rec_dict[key] = 'No reviews found'
                        print "Couldn't find reviews on " + str(key)
            if menu not in menu_list: 
                menu_list.append((key, menu))
    return menu_list, rec_dict

In [165]:
db_dict = get_restos(minis)

In [20]:
len(db_dict)

50

In [21]:
db_dict.keys()

[560001,
 160001,
 510001,
 200001,
 250001,
 460001,
 410001,
 170001,
 480001,
 360001,
 570001,
 310001,
 550001,
 520001,
 100001,
 470001,
 190001,
 420001,
 580001,
 370001,
 110001,
 320001,
 530001,
 270001,
 380001,
 220001,
 120001,
 260001,
 590001,
 330001,
 540001,
 130001,
 280001,
 490001,
 230001,
 440001,
 180001,
 390001,
 300001,
 340001,
 210001,
 430001,
 290001,
 500001,
 240001,
 150001,
 450001,
 400001,
 140001,
 350001]

In [249]:
menus, recs_db = recs_collector(db_dict)

screlped Hiro-Sushi-Scottsdale
screlped Zinc-Bistro-Scottsdale
screlped Oreganos-Pizza-Bistro-Scottsdale
Houstons-Restaurant-Scottsdale Menu not on Yelp
screlped Houstons-Restaurant-Scottsdale
Lox,-Stock-&-Bagel-Scottsdale Menu not on Yelp
screlped Lox,-Stock-and-Bagel-Scottsdale
screlped RA-Sushi-Bar-Restaurant-Scottsdale
Ristorante-Giuseppe-Scottsdale Menu not on Yelp
screlped Ristorante-Giuseppe-Scottsdale
screlped Flos-Scottsdale
screlped The-Melting-Pot-Scottsdale
screlped The-Cheesecake-Factory-Scottsdale
[u'want', u'hot', u'place', u'scottsdale', 'index_0', u'wasn', u'service', u'came', u'busy', u'rice', u'best', u'asked', u'things', u'dining', u'brought', u'right', u'people', u'happy', u'love', u'awesome', u'didn', u'wrong', u'tasted', u'look', u'say', u'calamari', u'slow', u'ginger', u'potstickers', u'favorite', u'dressing', u'large', u'small', u'experience', u'salty', u'sauce', u'really', u'entrees', u'wraps', u'location', u'fresh', u'try', 'index', u'let', u'great', u'menu',

In [325]:
menus, recs_db = recs_collector(db_dict, recs_db, menus, start=140001, num_dfs=2)

Pho-Kim-Long-Las-Vegas Menu not on Yelp
screlped Pho-Kim-Long-Las-Vegas
Ichiza-Las-Vegas Menu not on Yelp
screlped Ichiza-Las-Vegas
Pho-So-1-Las-Vegas Menu not on Yelp
screlped Pho-So-1-Las-Vegas
Omelet-House-Las-Vegas Menu not on Yelp
screlped Omelet-House-Las-Vegas
Tofu-Hut-Las-Vegas Menu not on Yelp
screlped Tofu-Hut-Las-Vegas
Golden-Steer-Las-Vegas Menu not on Yelp
screlped Golden-Steer-Las-Vegas
Sam-Woo-BBQ-Restaurant-Las-Vegas Menu not on Yelp
screlped Sam-Woo-BBQ-Restaurant-Las-Vegas
Aloha-Kitchen-&-Bar-Las-Vegas Menu not on Yelp
screlped Aloha-Kitchen-and-Bar-Las-Vegas
Capriottis-Sandwich-Shop-Las-Vegas Menu not on Yelp
screlped Capriottis-Sandwich-Shop-Las-Vegas
I-Love-Sushi-Henderson Menu not on Yelp
screlped I-Love-Sushi-Henderson
Komol-Restaurant-Las-Vegas Menu not on Yelp
screlped Komol-Restaurant-Las-Vegas
Tea-Station-Las-Vegas Menu not on Yelp
screlped Tea-Station-Las-Vegas
Island-Style-Las-Vegas Menu not on Yelp
screlped Island-Style-Las-Vegas
Joyful-House-Chinese-Cuisi

In [291]:
count = 0
missings = []
rec_list = []
for key, value in recs_db.iteritems():
    if type(value)== str:
        missings.append(key)
        count += 1
    else:
        rec_list.append(key)
len(rec_list)

24

In [281]:
rec_list

[u'Oreganos-Pizza-Bistro-Scottsdale',
 u'Zinc-Bistro-Scottsdale',
 u'YCs-Mongolian-Grill-Tempe',
 u'Teds-Hot-Dogs-Tempe',
 u'Tokyo-Lobby-Sushi-and-Grill-Glendale',
 u'The-Cheesecake-Factory-Peoria',
 u'Sakana-Sushi-&-Grill-Glendale',
 u'Flos-Scottsdale',
 u'Venezias-New-York-Style-Pizzeria-Tempe',
 u'The-Cheesecake-Factory-Scottsdale',
 u'Babbo-Italian-Eatery-Glendale',
 u'Arriba-Mexican-Grill-Scottsdale',
 u'Wildflower-Bread-Company-Scottsdale',
 u'Thee-Pitts-Again-Glendale',
 u'Hiro-Sushi-Scottsdale',
 u'Rock-Springs-Cafe-&-General-Store-Black-Canyon-City',
 u'Pinnacle-Peak-Patio-Scottsdale',
 u'The-Melting-Pot-Scottsdale',
 u'RA-Sushi-Bar-Restaurant-Scottsdale']

In [27]:
with open('att_dict.pkl', 'r') as picklefile:
     atts = pickle.load(picklefile)

In [311]:
rlist = rec_list

In [320]:
resto = rlist[10]
print resto
atts[resto]

Flos-Scottsdale


{'cats': [u'Thai', u'Asian Fusion', u'Chinese', u'Restaurants'],
 'city': u'Scottsdale',
 'name': u"Flo's",
 'num_reviews': 157,
 'rating': 3.5}

In [321]:
recs_db[resto][0]

,MenuItem,Trendiness,Buzz,Score,Controversy
5,Lettuce Cups,10.000,9,7.500,3.086
9,Springs Roll,7.447,9,6.389,5.258
2,Crab Puffs,6.478,5,7.000,10.000
7,Pad Thai,6.478,8,6.250,4.630
0,Cashew Chicken,5.229,8,6.875,4.051
1,Chicken & Curry Sauces,4.437,11,6.027,2.158
6,Orange Chicken,3.468,5,6.000,2.963
8,Pot Stickers,3.468,15,5.625,4.180
4,Hot & Sour,3.468,27,5.556,6.973
3,Garlic Chicken Stir-fry,0.458,5,5.500,6.296


In [98]:
recs_db[resto][0].to_dict()

{'Buzz': {u'"Mac and Cheese"': 23,
  u'2 Mussels': 16,
  u'Crispy Duck Confit, Pickled Kumquats, Foie Gras Aioli': 48,
  u'Croque Monsieur, Toasted Brioche, French Ham': 48,
  u'Dungeness Crab Crepe, Sweet Lobster Cream': 10,
  u'Grilled Curry Chicken Salad Pumpernickel Sandwich': 35,
  u'Local Organic Savoy Spinach and Frisee Salad*': 8,
  u'Onion Soup Gratinee': 75,
  u'Oysters': 11,
  u'Prime Flat Iron Steak and Iceberg Salad*': 63,
  u'Seared Scottish Salmon, White Bean Parmesan Fondue*': 6,
  u'Truffle and Parmesan Frites': 25,
  u'Zinc Burger, Bacon Blue Cheese or Truffled Gruyere*': 7,
  u'Zinc Valrhona Chocolate Souffle': 29},
 'Controversy': {u'"Mac and Cheese"': 3.0722370915579349,
  u'2 Mussels': 5.0136612021857925,
  u'Crispy Duck Confit, Pickled Kumquats, Foie Gras Aioli': 4.6098529486083786,
  u'Croque Monsieur, Toasted Brioche, French Ham': 3.8776010556110534,
  u'Dungeness Crab Crepe, Sweet Lobster Cream': 1.958469945355191,
  u'Grilled Curry Chicken Salad Pumpernickel 

In [104]:
recs = {}
for resto in rec_list:
    recs[resto] = recs_db[resto][0].T.to_dict()

In [105]:
pprint(recs)

{u'Arriba-Mexican-Grill-Scottsdale': {u'Pecos Sour Cream Enchiladas': {'Buzz': 12.0,
                                                                       'Controversy': 9.9444444444444464,
                                                                       'Score': 3.5416666666666661,
                                                                       'Trendiness': -7.7777777777777777},
                                      u'White Sands Chimichanga': {'Buzz': 15.0,
                                                                   'Controversy': 8.1020408163265305,
                                                                   'Score': 7.2321428571428568,
                                                                   'Trendiness': 7.2222222222222223}},
 u'Flos-Scottsdale': {u'Hot & Sour': {'Buzz': 27.0,
                                      'Controversy': 10.0,
                                      'Score': 5.5555555555555545,
                                      'Tre

In [129]:
with open('recs.pkl', 'w') as picklefile:
     pickle.dump(recs, picklefile)

In [120]:
for resto, rec in recs.iteritems():
    menu = {}
    for name, ratings in rec.iteritems():
        print name, ratings
        scores = {}
        scores['Trendiness'] = ratings['Trendiness']
        scores['Buzz']=ratings['Buzz']
        scores['Score']=ratings['Score']
        scores['Controversy'] = ratings['Controversy']
        courses[name] = scores
    menu[resto] = courses
    restos.insert(menu)
    print 'inserted'

Lettuce Cups {'Trendiness': 10.0, 'Score': 7.5, 'Controversy': 4.4262295081967205, 'Buzz': 9.0}
Hot & Sour {'Trendiness': 2.2222222222222223, 'Score': 5.5555555555555545, 'Controversy': 10.0, 'Buzz': 27.0}
Pot Stickers {'Trendiness': 3.3333333333333335, 'Score': 5.8333333333333339, 'Controversy': 8.8524590163934409, 'Buzz': 9.0}
Orange Chicken {'Trendiness': 4.4444444444444446, 'Score': 6.4285714285714279, 'Controversy': 1.6259618601538974, 'Buzz': 7.0}
Springs Roll {'Trendiness': 5.5555555555555554, 'Score': 6.3888888888888884, 'Controversy': 7.5409836065573748, 'Buzz': 9.0}
Pad Thai {'Trendiness': 4.4444444444444446, 'Score': 6.25, 'Controversy': 6.6393442622950811, 'Buzz': 8.0}


ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused

In [179]:
with open('rec_db0.pkl', 'w') as picklefile:
     pickle.dump(recs_db, picklefile)